In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torchdiffeq import odeint
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

# ================== Configuration Parameters ==================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
image_size = 28
channels = 1
batch_size = 128
lr = 1e-4
epochs = 100
num_classes = 10
model_save_path = 'FMmodel.pth'

# ================== Data Loading ==================

# Normalize image to [-1,1]
def normalize_img(x):
    return 2 * x - 1

transform = transforms.Compose([
    transforms.ToTensor(),              # Convert image to tensor
    transforms.Lambda(normalize_img)    # Apply normalization
])

# ================== Model Architecture ==================
class ConditionedDoubleConv(nn.Module):
    """Double convolution module with condition injection"""

    def __init__(self, in_channels, out_channels, cond_dim):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.norm1 = nn.GroupNorm(8, out_channels)
        self.conv2 = nn.Conv2d(out_channels + cond_dim, out_channels, kernel_size=3, padding=1)
        self.norm2 = nn.GroupNorm(8, out_channels)

    def forward(self, x, cond):
        x = F.silu(self.norm1(self.conv1(x)))
        cond = cond.expand(-1, -1, x.size(2), x.size(3))  # Dynamic condition broadcasting
        x = torch.cat([x, cond], dim=1)
        return F.silu(self.norm2(self.conv2(x)))


class Down(nn.Module):
    """Downsampling module"""

    def __init__(self, in_channels, out_channels, cond_dim):
        super().__init__()
        self.maxpool = nn.MaxPool2d(2)
        self.conv = ConditionedDoubleConv(in_channels, out_channels, cond_dim)

    def forward(self, x, cond):
        x = self.maxpool(x)
        return self.conv(x, cond)


class Up(nn.Module):
    """Upsampling module"""

    def __init__(self, in_channels, out_channels, cond_dim):
        super().__init__()
        self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.conv = ConditionedDoubleConv(in_channels, out_channels, cond_dim)

    def forward(self, x1, x2, cond):
        x1 = self.up(x1)
        # Size alignment
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]
        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x, cond)


class ConditionalUNet(nn.Module):
    """Dimension-safe conditional UNet"""

    def __init__(self):
        super().__init__()
        # Unified condition encoding dimensions
        self.t_dim = 16
        self.label_dim = 16
        self.cond_dim = self.t_dim + self.label_dim  # 32

        # Time embedding
        self.time_embed = nn.Sequential(
            nn.Linear(1, 32),
            nn.SiLU(),
            nn.Linear(32, self.t_dim)
        )
        # Label embedding
        self.label_embed = nn.Embedding(num_classes, self.label_dim)

        # Encoder path
        self.inc = ConditionedDoubleConv(1, 64, self.cond_dim)
        self.down1 = Down(64, 128, self.cond_dim)
        self.down2 = Down(128, 256, self.cond_dim)

        # Decoder path
        self.up1 = Up(256 + 128, 128, self.cond_dim)  # Input channel correction
        self.up2 = Up(128 + 64, 64, self.cond_dim)
        self.outc = nn.Conv2d(64, 1, kernel_size=1)

    def forward(self, x, t, labels):
        # Condition encoding (unified dimensions)
        t_emb = self.time_embed(t.view(-1, 1))  # [B, 16]
        lbl_emb = self.label_embed(labels)  # [B, 16]
        cond = torch.cat([t_emb, lbl_emb], dim=1)  # [B, 32]
        cond = cond.unsqueeze(-1).unsqueeze(-1)  # [B, 32, 1, 1]

        # Encoder
        x1 = self.inc(x, cond)
        x2 = self.down1(x1, cond)
        x3 = self.down2(x2, cond)

        # Decoder
        x = self.up1(x3, x2, cond)
        x = self.up2(x, x1, cond)
        return self.outc(x)


# ================== Training and Generation ==================
# Initialize model and optimizer here for global access (especially for generate_with_label)
model = ConditionalUNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)


@torch.no_grad()
def generate_with_label(label, num_samples=16):
    """
    Generate samples with specified label.
    Args:
        label (int): Digit label to generate (0-9).
        num_samples (int): Number of samples to generate.
    Returns:
        torch.Tensor: Generated image tensor, shape (num_samples, image_size, image_size), values in [0, 1].
    """
    # Save current model training state and set to eval mode
    current_model_state = model.training
    model.eval()

    # Create initial noise and label tensors
    x0 = torch.randn(num_samples, 1, image_size, image_size, device=device)
    labels = torch.full((num_samples,), label, device=device, dtype=torch.long)

    # ODE: t is time, x is current state
    def ode_func(t: torch.Tensor, x: torch.Tensor):
        t_expanded = t.expand(x.size(0))  # [1] -> [num_samples]
        vt = model(x, t_expanded, labels)  # Predict velocity field
        return vt

    # Time points: 0 -> 1
    t_eval = torch.tensor([0.0, 1.0], device=device)

    # Solve ODE (adaptive step size)
    generated = odeint(
        ode_func,
        x0,
        t_eval,
        rtol=1e-5,
        atol=1e-5,
        method='dopri5'
    )

    # Restore training state
    model.train(current_model_state)

    # Post-processing
    images = (generated[-1].clamp(-1, 1) + 1) / 2  # [-1,1] -> [0,1]
    return images.cpu().squeeze(1)


def visualize_train(epoch):
    """
    Generate visualization using current model:
    Create a 10x10 grid with columns 1-10 for each digit (0-9),
    generating 10 images per digit with labels in the first row.
    """
    print("Generating training visualization samples...")
    plt.figure(figsize=(10, 10))
    plt.subplots_adjust(wspace=0.05, hspace=0.05)  # Reduce subplot spacing

    # Generate 10 images for each digit 0-9
    for label in range(num_classes):
        # Generate 10 samples for current digit
        generated_images = generate_with_label(
            label=label,
            num_samples=10
        ).numpy()  # Shape (10, 28, 28)

        # Plot in current column, ensuring each column represents one digit
        for i in range(10):
            # Subplot position calculation: (row_index * total_columns) + column_index + 1
            # We want column 0 for digit 0, column 1 for digit 1, etc.
            # So: (row i * num_classes (10)) + column label + 1
            ax = plt.subplot(10, num_classes, (i * num_classes) + label + 1)
            plt.imshow(generated_images[i], cmap='gray', vmin=0, vmax=1)
            ax.axis('off')
            # Add digit label in first row of each column (when i == 0)
            if i == 0:
                ax.set_title(str(label), fontsize=16, pad=5)  # set_title is more appropriate
    plt.suptitle("Generated Samples During Training", fontsize=20, y=0.97)
    plt.savefig(f"epoch{epoch}.jpg")
    plt.close()


def hundred_image(model_path=model_save_path, num_sample=5):
    print(f"Loading model from {model_path} and generating {num_sample} images...")
    # Create new model instance and load weights
    global model
    original_model = model  # Save original model reference

    loaded_model = ConditionalUNet().to(device)
    if os.path.exists(model_path):
        loaded_model.load_state_dict(torch.load(model_path, map_location=device))
        loaded_model.eval()  # Evaluation mode
    else:
        print(f"Error: Model file not found: {model_path}. Cannot generate images.")
        return

    model = loaded_model  # Temporarily replace global model with loaded model

    for k in range(num_sample):
        plt.figure(figsize=(10, 10))
        plt.subplots_adjust(wspace=0.05, hspace=0.05)

        print(f"Generating image {k + 1}...")
        for label in tqdm(range(num_classes), desc="Generating images for each digit"):
            # Generate 10 samples for current digit
            generated_images = generate_with_label(
                label=label,
                num_samples=10
            ).numpy()  # Shape (10, 28, 28)

            # Plot in current column
            for i in range(10):
                ax = plt.subplot(10, num_classes, (i * num_classes) + label + 1)
                plt.imshow(generated_images[i], cmap='gray', vmin=0, vmax=1)
                ax.axis('off')
                # Add label in first row
                if i == 0:
                    ax.set_title(str(label), fontsize=16, pad=5)

        plt.suptitle(f"Final Generated Samples (Generation {k + 1})", fontsize=20, y=0.97)
        plt.savefig(f"generated_image{k + 1}.jpg")
        print(f"Generated image saved to: generated_image{k + 1}.jpg")
        plt.close()

    # Restore global model reference
    model = original_model


def train(num_epochs=100):
    """Training loop"""
    print("Starting training...")
    # Ensure train_loader is available here since it's initialized in the if __name__ == "__main__": block
    global train_loader

    for epoch in range(num_epochs):
        # Use tqdm to show training progress bar
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch + 1}")
        model.train()  # Training mode
        total_loss = 0

        for images, labels in progress_bar:  # Add progress bar
            images = images.to(device)
            labels = labels.to(device)

            # Dynamic noise generation
            noise = torch.randn_like(images)
            t = torch.rand(images.size(0), device=device)
            # Flow Matching target velocity is the velocity field from noise x0 to real data x1
            # x_t = (1-t) * x0 + t * x1
            xt = (1 - t.view(-1, 1, 1, 1)) * noise + t.view(-1, 1, 1, 1) * images

            # Forward pass, model predicts velocity field v_t
            vt_pred = model(xt, t, labels)
            # True velocity field v_t = x1 - x0
            loss = F.mse_loss(vt_pred, images - noise)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Gradient clipping to prevent explosion
            optimizer.step()

            total_loss += loss.item()

            # Update progress bar with loss display
            progress_bar.set_postfix({"Loss": f"{total_loss:.4f}"})

        # Generate samples every 10 epochs
        if (epoch + 1) % 10 == 0:
            visualize_train(epoch + 1)
        # Generate sample for first epoch
        if epoch == 0:
            visualize_train(epoch + 1)

    # Save model after training
    torch.save(model.state_dict(), model_save_path)
    print(f"Training complete. Model saved to: {model_save_path}")


if __name__ == "__main__":
    train_dataset = torchvision.datasets.MNIST(
        root='./data', train=True, download=True, transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                              num_workers=2)
    # Comment out when using existing model
    train(epochs)
    hundred_image(model_save_path)

Starting training...


Epoch 1: 100%|██████████| 469/469 [00:51<00:00,  9.03it/s, Loss=144.0164]


Generating training visualization samples...


Epoch 2:  47%|████▋     | 220/469 [00:24<00:27,  8.94it/s, Loss=46.2330]